In [ ]:
import pandas as pd
import numpy as np

# set seed for reproducibility
np.random.seed(0) 

In [ ]:
folderpath = "/opt/helthcare-final-project-autism/notebooks/data/processed"
filepath = f"{folderpath}/2017-2018_NSCH_DRC.csv.gzip"

In [ ]:
df = pd.read_csv(filepath, index_col='HHID', compression='gzip')
have_autism = df[['K2Q35B']].fillna(2)
df = df[[col for col in df.columns.values if col != 'K2Q35B']]

In [ ]:
from scipy.stats import spearmanr
cor_col = []
p_value_col = []
for col in df.columns.values:
    cor, p_value = spearmanr(df[col].values, have_autism.values, nan_policy='omit')
    cor_col.append(cor)
    p_value_col.append(cor)
    print(f"{col} - spearman: {cor} - p-value: {p_value}")

In [ ]:
df_variables = pd.read_csv(f"{folderpath}/2017-2018_description_columns.csv", index_col="column")

In [ ]:
df_spearman_corr = pd.DataFrame({'spearman': cor_col,
                                 'p_value': p_value_col},
                                 index=df.columns.values)
df_spearman_corr = df_spearman_corr.sort_values(by=['spearman'], ascending=False)

In [ ]:
df_spearman_corr = df_spearman_corr.join(df_variables)
df_spearman_corr.head(n=50)

In [ ]:
# itens_list = ['_1718']
# df_spearman_corr_[df_spearman_corr_.index.str.contains(f"({'|'.join(itens_list)})")]

description_itens_list = ['screener']
exclude_columns = df_spearman_corr[df_spearman_corr.description.fillna('').str.contains(f"({'|'.join(description_itens_list)})", case=False)].index.values
df_spearman_corr_ = df_spearman_corr.loc[[c for c in df_spearman_corr.index.values if c not in exclude_columns]]

df_spearman_corr_.head(n=25)

In [ ]:
selected_columns = (df_spearman_corr_.head(n=25).index.values.tolist())
df_to_model = df[selected_columns]
df_to_model.head()
# have_autism # to predict

In [ ]:
y = have_autism.values
X = df_to_model.fillna('999').values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(X_test)))